# Assignment 1: Dimensionality Reduction

The implementations in this notebook were generated using AI. Some functions may be incorrectly implemented, some may not be efficient.

Your goals are:
- Fix all bugs
- Improve efficiency
- Complete the missing analysis

**IMPORTANT**: Do not change the function names or the function parameters, as they will be used by the grading script.

In [1]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import time

## Data Loading

In [2]:
# Load the data
# You can use numpy to load the data from a file.

def load_data(file_path):
    """
    Load the dataset from a given file path.
    Args:
        file_path: str, path to the data file
    Returns:
        data: numpy array of shape (n_samples, n_features)
    """
    data = np.loadtxt(file_path)
    return data

# Example usage:
data = load_data('./data/Asgmnt1_data.txt')
print('Data shape:', data.shape)

Data shape: (16000, 128)


## Data Normalization
Perform row-wise normalization on the input data.

In [3]:
## Normalization
def normalize_data(X):
    """
    Normalize the data matrix X using z-score normalization.
    Args:
        X: numpy array of shape (n_samples, n_features)
    Returns:
        X_normalized: numpy array of shape (n_samples, n_features)
    """
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    # Avoid division by zero
    std[std == 0] = 1
    X_normalized = (X - mean) / std
    return X_normalized

# Example usage:
data_norm = normalize_data(data)
print('Normalized data shape:', data_norm.shape)

Normalized data shape: (16000, 128)


## Euclidean Distance Matrix

Implement the function below to compute the Euclidean distance matrix for a given data matrix.


In [4]:
def euclidean_distance_matrix(X):
    """
    Compute the pairwise Euclidean distance matrix for X.
    Args:
        X: numpy array of shape (n_samples, n_features)
    Returns:
        dist_matrix: numpy array of shape (n_samples, n_samples)
    """
    n_samples = X.shape[0]
    dist_matrix = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(i + 1, n_samples):
            dist = np.sqrt(np.sum((X[i] - X[j]) ** 2))
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist  # symmetric
    return dist_matrix

# Example usage (you can uncomment this after implementing for testing):
dist_matrix = euclidean_distance_matrix(data_norm[:100])
print('Distance matrix shape:', dist_matrix.shape)


Distance matrix shape: (100, 100)


## Wavelet Transform (Haar)

Implement a function to generate a Haar matrix and use it to perform wavelet transform.

You need to use the first 4 coefficients to perform the Wavelet transform.


In [16]:
def haar_matrix(n: int):
    """
    Generate an n x n Haar matrix.
    Args:
        n: int, size of the Haar matrix (must be a power of 2)
    Returns:
        H: numpy array of shape (n, n)
    """
    if n < 1 or (n & (n - 1)) != 0:
        raise ValueError("n must be a power of 2 and >= 1")
    H = np.array([[1.0]])
    while H.shape[0] < n:
        H = np.kron(H, [[1, 1], [1, -1]])
    # H = H / np.sqrt(n) # Normalize the Haar matrix
    return H


# Example usage:
H = haar_matrix(8)
print(H)

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1. -1.  1. -1.  1. -1.  1. -1.]
 [ 1.  1. -1. -1.  1.  1. -1. -1.]
 [ 1. -1. -1.  1.  1. -1. -1.  1.]
 [ 1.  1.  1.  1. -1. -1. -1. -1.]
 [ 1. -1.  1. -1. -1.  1. -1.  1.]
 [ 1.  1. -1. -1. -1. -1.  1.  1.]
 [ 1. -1. -1.  1. -1.  1.  1. -1.]]


In [19]:
def wavelet_transform(X, haar_n: int, reduced_n: int):
    """
    Apply Haar wavelet transform to X using an n x n Haar matrix, and reduce the number of features to `reduced_n`.

    This function should call haar_matrix to generate the Haar matrix.

    Args:
        X: numpy array of shape (n_samples, n_features)
        haar_n: int, number of features to be passed to haar_matrix()
        reduced_n: int, number of features to keep after wavelet transform
    Returns:
        X_wavelet: numpy array of shape (n_samples, reduced_n)
    """
    # Generate Haar matrix (with 0, 1, -1 entries)
    H = haar_matrix(haar_n)
    # Apply the transform: project data onto Haar basis
    X_wavelet_full = np.dot(X, H.T)
    # Keep only the first reduced_n features
    X_wavelet = X_wavelet_full[:, :reduced_n]
    return X_wavelet

# Example usage:
data_wavelet = wavelet_transform(data_norm, data_norm.shape[1], reduced_n=4)
print('Wavelet-transformed data shape:', data_wavelet.shape)

Wavelet-transformed data shape: (16000, 4)


## Principal Component Analysis (PCA)

Implement PCA without using any external libraries like scikit-learn. Please keep the BEST 4  principal components.


In [20]:
def pca(X, n_components: int):
    """
    Perform PCA on X and return the projected data with the top n_components.
    Args:
        X: numpy array of shape (n_samples, n_features)
        n_components: int, number of principal components to keep
    Returns:
        X_pca: numpy array of shape (n_samples, n_components)
    """
    # Center the data
    X_centered = X - np.mean(X, axis=0)
    # Compute covariance matrix
    cov = np.cov(X_centered, rowvar=False)
    # Eigen decomposition
    eigvals, eigvecs = np.linalg.eigh(cov)
    # Sort eigenvectors by eigenvalues in descending order
    idx = np.argsort(eigvals)[::-1]
    eigvecs = eigvecs[:, idx]
    # Select the top n_components
    components = eigvecs[:, :n_components]
    # Project the data
    X_pca = np.dot(X_centered, components)
    return X_pca

# Example usage:
data_pca = pca(data_norm, 4)
print('PCA-transformed data shape:', data_pca.shape)

PCA-transformed data shape: (16000, 4)


## Benchmarking and Analysis

Compare the dimensionality reduction techniques.

Please ensure that your analysis is well-structured and well-documented. You can use use plots to visualize the comparison results.
For visualizing high dimensional data, you can use t-SNE (from `scikit-learn`, [link here](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)) to reduce the dimensions to 2D or 3D for plotting.


### 1. Elasped Time

Compare the elapsed time for each dimensionality reduction technique. You can use the `time` module to measure the execution time of each function.


In [22]:
# Your implementation for measuring elapsed time goes here
import time

# Use a subset for demonstration if the dataset is large (e.g., first 100 samples)
subset = 1000
X_orig = data_norm[:subset]
X_wavelet = wavelet_transform(X_orig, haar_n=X_orig.shape[1], reduced_n=4)
X_pca = pca(X_orig, n_components=4)

# 1. Time for original dataset
start = time.time()
dist_orig = euclidean_distance_matrix(X_orig)
time_orig = time.time() - start
print(f"Time for Euclidean distance (original): {time_orig:.4f} seconds")

# 2. Time for wavelet-transformed dataset
start = time.time()
dist_wavelet = euclidean_distance_matrix(X_wavelet)
time_wavelet = time.time() - start
print(f"Time for Euclidean distance (wavelet): {time_wavelet:.4f} seconds")

# 3. Time for PCA-transformed dataset
start = time.time()
dist_pca = euclidean_distance_matrix(X_pca)
time_pca = time.time() - start
print(f"Time for Euclidean distance (PCA): {time_pca:.4f} seconds")

Time for Euclidean distance (original): 1.0948 seconds
Time for Euclidean distance (wavelet): 1.0571 seconds
Time for Euclidean distance (PCA): 1.0334 seconds


### 2. Distance Relation Consistency
Compare the distance relation consistency of the original data and the reduced data. You can use the Euclidean distance matrix computed earlier to analyze this.

In [24]:
# Your implementation for distance relation consistency goes here
import random

def relationship_consistency(dist_orig, dist_reduced, num_samples=10000, seed=42):
    """
    Compare pairwise distance relationships between original and reduced distance matrices.
    Returns the fraction of relationships preserved.
    Args:
        dist_orig: numpy array, shape (n, n), original distance matrix
        dist_reduced: numpy array, shape (n, n), reduced distance matrix
        num_samples: int, number of random triplets to sample
        seed: int, random seed for reproducibility
    Returns:
        consistency: float, fraction of preserved relationships
    """
    np.random.seed(seed)
    n = dist_orig.shape[0]
    count = 0
    preserved = 0
    for _ in range(num_samples):
        # Randomly pick anchor, B, C (all different)
        a, b, c = np.random.choice(n, 3, replace=False)
        rel_orig = dist_orig[a, b] > dist_orig[a, c]
        rel_reduced = dist_reduced[a, b] > dist_reduced[a, c]
        if rel_orig == rel_reduced:
            preserved += 1
        count += 1
    return preserved / count

# Example usage:
cons_wavelet = relationship_consistency(dist_orig, dist_wavelet)
cons_pca = relationship_consistency(dist_orig, dist_pca)
print(f"Wavelet consistency: {cons_wavelet:.4f}")
print(f"PCA consistency: {cons_pca:.4f}")

Wavelet consistency: 0.4922
PCA consistency: 0.9961


### 3. Your Own Metric
With the help of AI tools, find another metric that can be used to compare the dimensionality reduction techniques.

In [3]:
# Code, plots, and analysis for your own metric goes here